# Automatic Model Selection

Test notebook for automatic model selection

In [1]:
import datetime
import logging
import sys
import numpy as np
import time
import math

from sklearn.preprocessing import StandardScaler, MinMaxScaler

from keras.callbacks import LearningRateScheduler
from keras.optimizers import Adam
import keras.backend as K

#sys.path.append('/Users/davidlaredorazo/Documents/University_of_California/Research/Projects')
sys.path.append('/media/controlslab/DATA/Projects')

import ann_framework.aux_functions as aux_functions

import automatic_model_selection
from automatic_model_selection import Configuration
from ann_encoding_rules import Layers
import fetch_to_keras
#from CMAPSAuxFunctions import TrainValTensorBoard

#Tunable model
from ann_framework.tunable_model.tunable_model import SequenceTunableModelRegression, SequenceTunableModelClassification

#Data handlers
from ann_framework.data_handlers.data_handler_CMAPSS import CMAPSSDataHandler
from ann_framework.data_handlers.data_handler_MNIST import MNISTDataHandler
from ann_framework.data_handlers.data_handler_CIFAR10 import CIFAR10DataHandler

learningRate_scheduler = LearningRateScheduler(aux_functions.step_decay)

size_scaler = 0.5

#Use same configuration for all experiments, just change some of the parameters

#Define some random paramaters for the creation of the configuration, this will change for each test model
architecture_type = Layers.FullyConnected
#architecture_type = Layers.Convolutional
problem_type = 2  #1 for regression, 2 for classification
output_shape = 10 #If regression applies, number of classes
input_shape = (784,)

config = Configuration(architecture_type, problem_type, input_shape, output_shape, pop_size=5, 
                       tournament_size=3, max_similar=3, epochs=5, cross_val=0.2, size_scaler=size_scaler,
                       max_generations=10, binary_selection=True, mutation_ratio=0.8, 
                       similarity_threshold=0.2, more_layers_prob=0.4, verbose_individuals=True, 
                       show_model=True, verbose_training=1)

Using TensorFlow backend.


## Given a model get the compiled model

In [2]:
def get_compiled_model(model, problem_type, optimizer_params=[]):
    """Obtain a keras compiled model"""
    
    #Shared parameters for the models
    optimizer = Adam(lr=0.01, beta_1=0.5)
    
    if problem_type == 1:
        lossFunction = "mean_squared_error"
        metrics = ["mse"]
    elif problem_type == 2:
        lossFunction = "categorical_crossentropy"
        metrics = ["accuracy"]
    else:
        print("Problem type not defined")
        model = None
        return
    
    #Create and compile the models
    model.compile(optimizer = optimizer, loss = lossFunction, metrics = metrics)
    
    return model


def create_tunable_model(model_genotype, problem_type, input_shape, data_handler, model_number):
    
    K.clear_session()
    
    model = fetch_to_keras.decode_genotype(model_genotype, problem_type, input_shape, 1)
    
    model = get_compiled_model(model, problem_type, optimizer_params=[])
    
    if problem_type == 1:
        tModel = SequenceTunableModelRegression('ModelReg_SN_'+str(model_number), model, lib_type='keras', data_handler=data_handler)
    else:
        tModel = SequenceTunableModelClassification('ModelClass_SN_'+str(model_number), model, lib_type='keras', data_handler=data_handler)
        
    return tModel

## Load the CMAPSS datahandler

In [3]:
def cmapss_dhandler():

    #Selected as per CNN paper
    features = ['T2', 'T24', 'T30', 'T50', 'P2', 'P15', 'P30', 'Nf', 'Nc', 'epr', 'Ps30', 'phi', 'NRf', 'NRc', 'BPR', 
    'farB', 'htBleed', 'Nf_dmd', 'PCNfR_dmd', 'W31', 'W32']
    selected_indices = np.array([2, 3, 4, 7, 8, 9, 11, 12, 13, 14, 15, 17, 20, 21])
    selected_features = list(features[i] for i in selected_indices-1)
    data_folder = '../CMAPSSData'

    window_size = 24
    window_stride = 1
    max_rul = 129

    dhandler_cmapss = CMAPSSDataHandler(data_folder, 1, selected_features, max_rul, window_size, window_stride)

    input_shape = (len(selected_features)*window_size, )

    return dhandler_cmapss, input_shape

## Function to save top models

In [4]:
def save_best_models(best_models_list, global_best_model_index, saveto, input_shape, data_handler, 
                     problem_type=1, data_scaler=None, train_epochs=100, metrics=[], round=0):
    
    n_models = len(best_models_list)
    
    for ind_model, i in zip(best_models_list, range(n_models)):
        
        tModel = create_tunable_model(ind_model.stringModel, problem_type, input_shape, data_handler, i)
        kmodel = tModel.model
        model_json = kmodel.to_json()
        
        #Save model's architecture
        string_append = str(i) if i != global_best_model_index else 'global'
        with open(saveto+"bestModel_"+string_append+".json", "w") as json_file:
            json_file.write(model_json)
            
    #Train the global best, model has to be recompiled
    ind_model = best_models_list[global_best_model_index]
    tModel = create_tunable_model(ind_model.stringModel, problem_type, input_shape, data_handler, n_models)
    
    print(tModel.model.summary())
    print(tModel.data_handler)
    
    if tModel.data_handler.data_scaler != None:
        print("Using data handler scaler")
    elif tModel.data_scaler != None:
        print("Using tModel scaler (Overriding data handler scaler)")
    else:
        print("No data scaling used")
    
    if data_scaler != None:
        tModel.data_handler.data_scaler = None
        tModel.data_scaler = data_scaler
        
    tModel.load_data(unroll=True, verbose=1, cross_validation_ratio=0)
    tModel.print_data()
    tModel.epochs = train_epochs

    tModel.train_model(verbose=1)
    
    tModel.evaluate_model(metrics, round=round)
    
    kmodel = tModel.model
            
    # serialize weights to HDF5
    kmodel.save_weights(saveto+"bestModel_global.h5")
    
    print("Saved models for dataset 1 to disk")

## Get global best model

In [5]:
def recompute_globals_fitness(best_models, size_scaler, problem_type):
    """It is necessary to recompute the fiteness of global models since they have differnt normalization factors"""

    #print("Before normalization")
    #automatic_model_selection.print_best(best_models)
    
    normalize_scores(best_models)
    
    #print("After normalization")
    #automatic_model_selection.print_best(best_models)
    
    global_best_index = compute_fitness(best_models, size_scaler, problem_type)
    
    print("Recomputed fitness")
    automatic_model_selection.print_best(best_models)
    print("Global best index")
    print(global_best_index)
    
    return global_best_index


def normalize_scores(best_models):
    
    pop_size = len(best_models)
    raw_scores = np.zeros((pop_size,))
    
    for i in range(pop_size):
        model = best_models[i]
        raw_scores[i] = model.raw_score
        
    normalization_factor = np.linalg.norm(raw_scores)
    normalized_scores = raw_scores/normalization_factor
    
    for i in range(pop_size):
        model = best_models[i]
        model.normalized_score = raw_scores[i]
    
    
def compute_fitness(best_models, size_scaler, problem_type):
    
    pop_size = len(best_models)
    
    global_best_index = 0
    
    for i in range(pop_size):
        
        round_up_to = 3

        #Round up to the first 3 digits before computing log                                                                                                                                                          
        rounding_scaler = 10**round_up_to
        trainable_count = round(best_models[i].raw_size/rounding_scaler)*rounding_scaler
        size_score = math.log10(trainable_count)

        scaled_score = best_models[i].normalized_score

        #For classification estimate the error which is between 0 and 1                                                                                                                   
        if problem_type == 2:
            metric_score = (1 - scaled_score)*10 #Multiply by 10 to have a better scaling. I still need to find an appropriate scaling                                                
        else:
            metric_score = scaled_score*10 #Multiply by 10 to have a better scaling. I still need to find an appropiate scaling                                                       
    
        metric_scaler = 1-size_scaler
        print("metric_scaler %f"%metric_scaler)
        print("size scaler %f"%size_scaler)
    
        #Scalarization of multiobjective version of the fitness function                                                                                                                  
        best_models[i].fitness = metric_scaler*metric_score + size_scaler*size_score
        
        if best_models[i].fitness < best_models[global_best_index].fitness:
            global_best_index = i
            
    return global_best_index

## Test on MNIST

In [6]:
def mnist_test(dhandler_mnist, size_scaler=0.5, total_experiments=3):

    """Input can be of 3 types, ANN (1), CNN (2) or RNN (3)"""
    architecture_type = Layers.FullyConnected
    #architecture_type = Layers.Convolutional
    problem_type = 2  #1 for regression, 2 for classification
    output_shape = 10 #If regression applies, number of classes
    input_shape = (784,)
    #input_shape = (28,28,1)

    """
    pop_size = 5
    tournament_size = 3
    max_similar = 3
    total_experiments = 5
    count_experiments = 0
    unroll = True
    """
    #total_experiments = 1
    count_experiments = 0
    unroll = True

    global_best_list = []
    global_best = None
    global_best_index = 0
    experiment_times = np.zeros((total_experiments,1))

    scaler = None

    t = datetime.datetime.now()
    
    #Clear logging facility before attempting to create log
    for handler in logging.root.handlers[:]:
        logging.root.removeHandler(handler)
    
    logging.basicConfig(filename='logs/nn_evolution_mnist_' + t.strftime('%m%d%Y%H%M%S') + '.log', level=logging.INFO, 
                            format='%(levelname)s:%(threadName)s:%(message)s', datefmt='%m/%d/%Y %H:%M:%S')

    #mnist datahandler
    #dHandler_mnist = MNISTDataHandler()

    config.architecture_type = architecture_type
    config.problem_type = problem_type
    config.input_shape = input_shape
    config.output_shape = output_shape
    config.size_scaler = size_scaler
    config.epochs=5

    """
    config = Configuration(architecture_type, problem_type, input_shape, output_shape, pop_size, 
                           tournament_size, max_similar, epochs=20, cross_val=0.2, size_scaler=size_scaler,
                           max_generations=10, binary_selection=True, mutation_ratio=0.4, 
                           similarity_threshold=0.2, more_layers_prob=0.8)
    """

    while count_experiments < total_experiments:
        print("Launching experiment {}".format(count_experiments+1))
        logging.info("Launching experiment {}".format(count_experiments+1))


        start = time.time()
        best = automatic_model_selection.run_experiment(config, dHandler_mnist, count_experiments + 1, unroll=unroll,
                                                        learningRate_scheduler=learningRate_scheduler, 
                                                        tModel_scaler=scaler)
        end = time.time()
        elapsed_time = (end-start)/60
        experiment_times[count_experiments] = elapsed_time
        print("Experiment time: {} minutes".format(elapsed_time))
        logging.info("Experiment time: {} minutes".format(elapsed_time))


        best.individual_label = count_experiments

        global_best_list.append(best)
        
        """
        if global_best == None:
            global_best = best
        else:
            if best.fitness < global_best.fitness:
                global_best = best
                global_best_index = count_experiments
        """

        count_experiments =  count_experiments + 1
    
    print("Recompute globals fitness")
    global_best_index = recompute_globals_fitness(global_best_list, config.size_scaler, config.problem_type)
    global_best = global_best_list[global_best_index]
    
    total_experiment_time = experiment_times.sum()

    print("Global best list\n")
    logging.info("Global best list\n")
    automatic_model_selection.print_best(global_best_list)

    print("Global best is\n")
    print(global_best)
    logging.info("Global best is\n")
    logging.info(global_best)

    print("Global time {}".format(experiment_times.sum()))
    logging.info("Global time {}".format(experiment_times.sum()))
    
    logging.shutdown()
    
    return global_best_list, global_best_index, total_experiment_time

## Test on CIFAR10

In [7]:
def cifar_test(dHandler_cifar, size_scaler=0.5, total_experiments=3):

    """Input can be of 3 types, ANN (1), CNN (2) or RNN (3)"""
    architecture_type = Layers.FullyConnected
    problem_type = 2  #1 for regression, 2 for classification
    output_shape = 10 #If regression applies, number of classes
    input_shape = (3072,)
    """
    pop_size = 5
    tournament_size = 3
    max_similar = 3
    """
    total_experiments = 5
    count_experiments = 0
    unroll = True

    global_best_list = []
    global_best = None
    global_best_index = 0

    scaler = None

    t = datetime.datetime.now()

    logging.basicConfig(filename='logs/nn_evolution_cifar10_' + t.strftime('%m%d%Y%H%M%S') + '.log', level=logging.INFO, 
                            format='%(levelname)s:%(threadName)s:%(message)s', datefmt='%m/%d/%Y %H:%M:%S')

    #mnist datahandler
    #dHandler_cifar = CIFAR10DataHandler()

    """
    config = Configuration(architecture_type, problem_type, input_shape, output_shape, pop_size, tournament_size, max_similar, 
                           epochs=5, cross_val=0.2, size_scaler=size_scaler, max_generations=10, binary_selection=True, 
                           mutation_ratio=0.4, similarity_threshold=0.2, more_layers_prob=0.8)
    """

    config.architecture_type = architecture_type
    config.problem_type = problem_type
    config.input_shape = input_shape
    config.output_shape = output_shape

    while count_experiments < total_experiments:
        print("Launching experiment {}".format(count_experiments+1))
        logging.info("Launching experiment {}".format(count_experiments+1))

        best = automatic_model_selection.run_experiment(config, dHandler_cifar, count_experiments + 1, unroll=unroll,
                                                        learningRate_scheduler=learningRate_scheduler, 
                                                        tModel_scaler=scaler, verbose_data=0)

        best.individual_label = count_experiments

        global_best_list.append(best)

        if global_best == None:
            global_best = best
        else:
            if best.fitness < global_best.fitness:
                global_best = best
                global_best_index = count_experiments

        count_experiments =  count_experiments + 1
        
    total_experiment_time = experiment_times.sum()

    print("Global best list\n")
    logging.info("Global best list\n")
    automatic_model_selection.print_best(global_best_list)

    print("Global best is\n")
    print(global_best)
    logging.info("Global best is\n")
    logging.info(global_best)
    
    return global_best_list, global_best_index, total_experiment_time

## Test on CMAPSS

In [8]:
def cmapss_test(dhandler_cmapss, input_shape, size_scaler=0.5, total_experiments=3):

    """Input can be of 3 types, ANN (1), CNN (2) or RNN (3)"""
    architecture_type = Layers.FullyConnected
    problem_type = 1  #1 for regression, 2 for classification
    output_shape = 1 #If regression applies, number of classes

    """
    pop_size = 5
    tournament_size = 3
    max_similar = 3
    """
    #total_experiments = 5
    count_experiments = 0
    unroll = True

    global_best_list = []
    global_best = None
    global_best_index = 0
    experiment_times = np.zeros((total_experiments,1))

    scaler = MinMaxScaler(feature_range=(-1, 1))

    t = datetime.datetime.now()

    logging.basicConfig(filename='logs/nn_evolution_cmapss_' + t.strftime('%m%d%Y%H%M%S') + '.log', level=logging.INFO, 
                            format='%(levelname)s:%(threadName)s:%(message)s', datefmt='%m/%d/%Y %H:%M:%S')

    #cmaps datahandler
    #dhandler_cmaps, input_shape = cmaps_dhandler()

    """
    config = Configuration(architecture_type, problem_type, input_shape, output_shape, pop_size, tournament_size, 
                           max_similar, epochs=5, cross_val=0.2, size_scaler=size_scaler, max_generations=10, 
                           binary_selection=True, mutation_ratio=0.4, similarity_threshold=0.2, more_layers_prob=0.8)
    """

    config.architecture_type = architecture_type
    config.problem_type = problem_type
    config.input_shape = input_shape
    config.output_shape = output_shape
    config.epochs=10

    while count_experiments < total_experiments:
        print("Launching experiment {}".format(count_experiments+1))
        logging.info("Launching experiment {}".format(count_experiments+1))

        start = time.time()
        best = automatic_model_selection.run_experiment(config, dhandler_cmapss, count_experiments + 1, unroll=unroll,
                                                        learningRate_scheduler=learningRate_scheduler, 
                                                        tModel_scaler=scaler)
        end = time.time()
        elapsed_time = (end-start)/60
        experiment_times[count_experiments] = elapsed_time
        print("Experiment time: {} minutes".format(elapsed_time))
        logging.info("Experiment time: {} minutes".format(elapsed_time))

        best.individual_label = count_experiments

        global_best_list.append(best)

        """
        if global_best == None:
            global_best = best
        else:
            if best.fitness < global_best.fitness:
                global_best = best
                global_best_index = count_experiments
        """

        count_experiments =  count_experiments + 1
        
    print("Recompute globals fitness")
    global_best_index = recompute_globals_fitness(global_best_list, config.size_scaler, config.problem_type)
    global_best = global_best_list[global_best_index]
        
    total_experiment_time = experiment_times.sum()

    print("Global best list\n")
    logging.info("Global best list\n")
    automatic_model_selection.print_best(global_best_list)

    print("Global best is\n")
    print(global_best)
    logging.info("Global best is\n")
    logging.info(global_best)

    print("Global time {}".format(total_experiment_time))
    logging.info("Global time {}".format(total_experiment_time))
    
    return global_best_list, global_best_index, total_experiment_time

## Perform tests

In [9]:

#alphas = [0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8]
alphas = [0.8]
experiments = 5

global_best_list = []
global_best_index = 0
total_experiment_time = []
total_experiment_time = 0
avg_experiment_time = 0

dHandler_mnist = MNISTDataHandler()

for size_scaler in alphas:

    print("Running for alpha={}".format(size_scaler))
    
    global_best_list, global_best_index, total_experiment_time = mnist_test(dHandler_mnist, 
                                                                            size_scaler=size_scaler, 
                                                                            total_experiments=experiments)
    
    avg_experiment_time = total_experiment_time/experiments
    
    print("Total experiment time {}".format(total_experiment_time))
    print("Avg experiment time {}".format(avg_experiment_time))
    
    save_best_models(global_best_list, global_best_index, 
                     'best_models/mnist/yulin/alpha{}/'.format(size_scaler), input_shape=input_shape, 
                     data_handler=dHandler_mnist, problem_type=problem_type, train_epochs=10)



Running for alpha=0.8
Launching experiment 1

Generation 1
launch new
True
gen similar
False
Fetching model 0 to keras
Evaluating model 0
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
in (Dense)                   (None, 560)               439600    
_________________________________________________________________
dense_1 (Dense)              (None, 152)               85272     
_________________________________________________________________
dropout_1 (Dropout)          (None, 152)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 10)                1530      
Total params: 526,402
Trainable params: 526,402
Non-trainable params: 0
_________________________________________________________________
None
Loading data for the first time
Reloading data due to parameter change
Loading data. Cross-Validation ratio 0.2


/home/controlslab/anaconda3/envs/tf_gpu/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2179: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


training with cv
Train on 48000 samples, validate on 12000 samples
Epoch 1/5
48000/48000 [==============================] - 1s 17us/step - loss: 1.3126 - acc: 0.5804 - val_loss: 0.5296 - val_acc: 0.8757
Epoch 2/5
48000/48000 [==============================] - 0s 5us/step - loss: 0.5310 - acc: 0.8496 - val_loss: 0.3524 - val_acc: 0.9046
Epoch 3/5
48000/48000 [==============================] - 0s 5us/step - loss: 0.4002 - acc: 0.8860 - val_loss: 0.2982 - val_acc: 0.9156
Epoch 4/5
48000/48000 [==============================] - 0s 5us/step - loss: 0.3463 - acc: 0.9001 - val_loss: 0.2660 - val_acc: 0.9236
Epoch 5/5
12000/12000 [==============================] - 0s 14us/step
Fetching model 1 to keras
Evaluating model 1
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
in (Dense)                   (None, 216)               169560    
_________________________________________________________________
dense_1 (Dens

12000/12000 [==============================] - 0s 15us/step
metric_scaler 0.200000
size scaler 0.800000
Individual 0 score/normalized score/size/fitness 0.9275833333333333/0.9275833333333333/526402/4.721621928656325
metric_scaler 0.200000
size scaler 0.800000
Individual 1 score/normalized score/size/fitness 0.96225/0.96225/424058/4.577392685274186
metric_scaler 0.200000
size scaler 0.800000
Individual 2 score/normalized score/size/fitness 0.9693333333333334/0.9693333333333334/553330/4.655513438377091
metric_scaler 0.200000
size scaler 0.800000
Individual 3 score/normalized score/size/fitness 0.94725/0.94725/220522/4.381013818948089
metric_scaler 0.200000
size scaler 0.800000
Individual 4 score/normalized score/size/fitness 0.9730833333333333/0.9730833333333333/2006898/5.095871231323322

Generating offsprings
Applying Mutation
Launch new generation?: True

Generation 2
launch new
True
gen similar
False
Fetching model 0 to keras
Evaluating model 0
________________________________________

Train on 48000 samples, validate on 12000 samples
Epoch 1/5
48000/48000 [==============================] - 0s 7us/step - loss: 0.4913 - acc: 0.8706 - val_loss: 0.2425 - val_acc: 0.9332
Epoch 2/5
48000/48000 [==============================] - 0s 4us/step - loss: 0.1926 - acc: 0.9441 - val_loss: 0.1727 - val_acc: 0.9502
Epoch 3/5
48000/48000 [==============================] - 0s 4us/step - loss: 0.1407 - acc: 0.9587 - val_loss: 0.1443 - val_acc: 0.9592
Epoch 4/5
48000/48000 [==============================] - 0s 4us/step - loss: 0.1090 - acc: 0.9683 - val_loss: 0.1240 - val_acc: 0.9632
Epoch 5/5
12000/12000 [==============================] - 0s 13us/step
metric_scaler 0.200000
size scaler 0.800000
Individual 0 score/normalized score/size/fitness 0.9500833333333333/0.9500833333333333/616930/4.732061464559927
metric_scaler 0.200000
size scaler 0.800000
Individual 1 score/normalized score/size/fitness 0.9635/0.9635/248050/4.388561344660973
metric_scaler 0.200000
size scaler 0.800000
Individu

...

Evaluating model 0
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
in (Dense)                   (None, 24)                18840     
_________________________________________________________________
dropout_1 (Dropout)          (None, 24)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                250       
Total params: 19,090
Trainable params: 19,090
Non-trainable params: 0
_________________________________________________________________
None
Using previously loaded data
training with cv
Train on 48000 samples, validate on 12000 samples
Epoch 1/5
48000/48000 [==============================] - 0s 7us/step - loss: 1.9613 - acc: 0.3323 - val_loss: 1.4552 - val_acc: 0.7547
Epoch 2/5
48000/48000 [==============================] - 0s 4us/step - loss: 1.4246 - acc: 0.5657 - val_loss: 1.0790 - val_acc: 0.8337
Epoch 3/5
48

Evaluating model 0
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
in (Dense)                   (None, 24)                18840     
_________________________________________________________________
dense_1 (Dense)              (None, 736)               18400     
_________________________________________________________________
dense_2 (Dense)              (None, 10)                7370      
Total params: 44,610
Trainable params: 44,610
Non-trainable params: 0
_________________________________________________________________
None
Using previously loaded data
training with cv
Train on 48000 samples, validate on 12000 samples
Epoch 1/5
48000/48000 [==============================] - 0s 7us/step - loss: 0.7008 - acc: 0.8138 - val_loss: 0.3381 - val_acc: 0.9017
Epoch 2/5
48000/48000 [==============================] - 0s 4us/step - loss: 0.2750 - acc: 0.9200 - val_loss: 0.2668 - val_acc: 0.9205
Epoch 3/5
48


Generating offsprings
Applying Mutation
Launch new generation?: True

Generation 8
launch new
True
gen similar
False
Fetching model 0 to keras
Evaluating model 0
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
in (Dense)                   (None, 232)               182120    
_________________________________________________________________
dropout_1 (Dropout)          (None, 232)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                2330      
Total params: 184,450
Trainable params: 184,450
Non-trainable params: 0
_________________________________________________________________
None
Using previously loaded data
training with cv
Train on 48000 samples, validate on 12000 samples
Epoch 1/5
48000/48000 [==============================] - 0s 7us/step - loss: 1.2158 - acc: 0.6331 - val_loss: 0.5855 - val_acc: 0.8720
E

Evaluating model 0
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
in (Dense)                   (None, 24)                18840     
_________________________________________________________________
dropout_1 (Dropout)          (None, 24)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                250       
Total params: 19,090
Trainable params: 19,090
Non-trainable params: 0
_________________________________________________________________
None
Using previously loaded data
training with cv
Train on 48000 samples, validate on 12000 samples
Epoch 1/5
48000/48000 [==============================] - 0s 7us/step - loss: 1.1623 - acc: 0.6985 - val_loss: 0.6515 - val_acc: 0.8586
Epoch 2/5
48000/48000 [==============================] - 0s 4us/step - loss: 0.6105 - acc: 0.8467 - val_loss: 0.4518 - val_acc: 0.8890
Epoch 3/5
48



String Model
[[<Layers.FullyConnected: 1>, 64, 2, 0, 0, 0, 0, 0], [<Layers.FullyConnected: 1>, 10, 3, 0, 0, 0, 0, 0]]
<Individual(label = '2' fitness = 3.8757, raw_score = 0.9452, normalized_score = 0.9452, raw_size = 50890)>
Checksum vector: [ 2. 74.  5.  0.  0.  0.  0.  0.]


String Model
[[<Layers.FullyConnected: 1>, 64, 2, 0, 0, 0, 0, 0], [<Layers.FullyConnected: 1>, 10, 3, 0, 0, 0, 0, 0]]
<Individual(label = '3' fitness = 3.8744, raw_score = 0.9458, normalized_score = 0.9458, raw_size = 50890)>
Checksum vector: [ 2. 74.  5.  0.  0.  0.  0.  0.]


String Model
[[<Layers.FullyConnected: 1>, 24, 1, 0, 0, 0, 0, 0], [<Layers.FullyConnected: 1>, 10, 3, 0, 0, 0, 0, 0]]
<Individual(label = '4' fitness = 3.5872, raw_score = 0.9179, normalized_score = 0.9179, raw_size = 19090)>
Checksum vector: [ 2. 34.  4.  0.  0.  0.  0.  0.]
Global best index
1
Global best list



String Model
[[<Layers.FullyConnected: 1>, 48, 2, 0, 0, 0, 0, 0], [<Layers.FullyConnected: 1>, 48, 2, 0, 0, 0, 0, 0], [<Lay

In [10]:
""""
dHandler_cifar = CIFAR10DataHandler()

global_best_list, global_best_index = cifar_test(dHandler_cifar, size_scaler=size_scaler, total_experiments=1)
"""

'"\ndHandler_cifar = CIFAR10DataHandler()\n\nglobal_best_list, global_best_index = cifar_test(dHandler_cifar, size_scaler=size_scaler, total_experiments=1)\n'

In [11]:
"""
#alphas = [0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8]
alphas = [0.6]
experiments = 5
problem_type = 1

global_best_list = []
global_best_index = 0
total_experiment_time = []
total_experiment_time = 0
avg_experiment_time = 0

scaler = MinMaxScaler(feature_range=(-1, 1))

dhandler_cmapss, input_shape = cmapss_dhandler()
dhandler_cmapss_for_best, input_shape = cmapss_dhandler()

for size_scaler in alphas:

    print("Running for alpha={}".format(size_scaler))
    
    global_best_list, global_best_index, total_experiment_time = cmapss_test(dhandler_cmapss=dhandler_cmapss, 
                                                                            input_shape=input_shape,
                                                                            size_scaler=size_scaler, 
                                                                            total_experiments=experiments)
    
    print(global_best_list)
    print(global_best_index)
    
    avg_experiment_time = total_experiment_time/experiments
    
    print("Total experiment time {}".format(total_experiment_time))
    print("Avg experiment time {}".format(avg_experiment_time))
    
    save_best_models(global_best_list, global_best_index, 
                     'best_models/cmapss/yulin/alpha{}/'.format(size_scaler), input_shape=input_shape, 
                     data_handler=dhandler_cmapss_for_best, problem_type=problem_type, train_epochs=100, 
                     data_scaler=scaler)
    
"""    

'\n#alphas = [0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8]\nalphas = [0.6]\nexperiments = 5\nproblem_type = 1\n\nglobal_best_list = []\nglobal_best_index = 0\ntotal_experiment_time = []\ntotal_experiment_time = 0\navg_experiment_time = 0\n\nscaler = MinMaxScaler(feature_range=(-1, 1))\n\ndhandler_cmapss, input_shape = cmapss_dhandler()\ndhandler_cmapss_for_best, input_shape = cmapss_dhandler()\n\nfor size_scaler in alphas:\n\n    print("Running for alpha={}".format(size_scaler))\n    \n    global_best_list, global_best_index, total_experiment_time = cmapss_test(dhandler_cmapss=dhandler_cmapss, \n                                                                            input_shape=input_shape,\n                                                                            size_scaler=size_scaler, \n                                                                            total_experiments=experiments)\n    \n    print(global_best_list)\n    print(global_best_index)\n    \n    avg_experiment_time

In [12]:
"""
save_best_models(global_best_list, global_best_index, 'best_models/mnist/alpha{}/scalarized_version/'.format(size_scaler), 
                 input_shape=input_shape, data_handler=dHandler_mnist, problem_type=problem_type, train_epochs=100)
"""

"\nsave_best_models(global_best_list, global_best_index, 'best_models/mnist/alpha{}/scalarized_version/'.format(size_scaler), \n                 input_shape=input_shape, data_handler=dHandler_mnist, problem_type=problem_type, train_epochs=100)\n"

In [13]:
"""
save_best_models(global_best_list, global_best_index, 'best_models/cifar10/alpha{}/version2'.format(size_scaler), 
                 input_shape=input_shape, data_handler=dHandler_cifar, problem_type=problem_type, train_epochs=100)
"""

"\nsave_best_models(global_best_list, global_best_index, 'best_models/cifar10/alpha{}/version2'.format(size_scaler), \n                 input_shape=input_shape, data_handler=dHandler_cifar, problem_type=problem_type, train_epochs=100)\n"

In [14]:
"""
save_best_models(global_best_list, global_best_index, 'best_models/cmapss/alpha{}/'.format(size_scaler), 
                 input_shape=input_shape, data_handler=dhandler_cmaps, problem_type=problem_type, train_epochs=100, 
                 data_scaler=scaler, metrics=['rhs', 'rmse'], round=2)
"""

"\nsave_best_models(global_best_list, global_best_index, 'best_models/cmapss/alpha{}/'.format(size_scaler), \n                 input_shape=input_shape, data_handler=dhandler_cmaps, problem_type=problem_type, train_epochs=100, \n                 data_scaler=scaler, metrics=['rhs', 'rmse'], round=2)\n"